In [ ]:
%pip install llama-index-multi-modal-llms-gemini
%pip install llama-index-vector-stores-qdrant
%pip install llama-index-embeddings-gemini
%pip install llama-index-llms-gemini

%pip install pymilvus
%pip install matplotlib
%pip install PIL

In [12]:
!pip install llama-index 'google-generativeai>=0.3.0' matplotlib qdrant_client


  Using cached grpcio-1.62.0-cp312-cp312-macosx_10_10_universal2.whl.metadata (4.0 kB)
Using cached grpcio-1.62.0-cp312-cp312-macosx_10_10_universal2.whl (10.0 MB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.60.0
    Uninstalling grpcio-1.60.0:
      Successfully uninstalled grpcio-1.60.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymilvus 2.3.6 requires grpcio<=1.60.0,>=1.49.1, but you have grpcio 1.62.0 which is incompatible.


In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.vector_stores.milvus import MilvusVectorStore

from pymilvus import connections

from PIL import Image
import matplotlib.pyplot as plt
import os
import logging
import sys

In [8]:
%env GOOGLE_API_KEY=

In [9]:
image_documents = SimpleDirectoryReader("./data/images").load_data()

In [10]:
# put your local directore here


openai_mm_llm = OpenAIMultiModal(
    model="gpt-4-vision-preview", api_key=OPENAI_API_TOKEN, max_new_tokens=1500
)

response_1 = openai_mm_llm.complete(
    prompt="Describe the images as an alternative text",
    image_documents=image_documents,
)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4-vision-preview` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
vector_store = MilvusVectorStore(dim=1536, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

In [ ]:
# initialize llm
llm = OpenAI(model="gpt-3.5-turbo-0613")

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool], llm=llm, verbose=True)

In [ ]:
# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b
multiply_tool = FunctionTool.from_defaults(fn=multiply)